In [8]:
import pandas as pd

# Path to your file
file_path = '/Users/yash./netflix_titles.csv'

# Load CSV
df = pd.read_csv(file_path)

# Quick overview
print(df.head())
print(df.info())
print(df.isnull().sum())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

In [9]:
import pandas as pd

# Fill missing categorical data
df['director'] = df['director'].fillna('Unknown')
df['cast'] = df['cast'].fillna('Unknown')
df['country'] = df['country'].fillna('Unknown')
df['rating'] = df['rating'].fillna('Not Rated')
df['duration'] = df['duration'].fillna('Unknown')

# Convert date_added to datetime
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Fill missing date_added with a placeholder (optional)
df['date_added'] = df['date_added'].fillna(pd.Timestamp('2000-01-01'))

# Extract year and month from date_added
df['added_year'] = df['date_added'].dt.year
df['added_month'] = df['date_added'].dt.month

# Verify cleaning
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8807 non-null   object        
 1   type          8807 non-null   object        
 2   title         8807 non-null   object        
 3   director      8807 non-null   object        
 4   cast          8807 non-null   object        
 5   country       8807 non-null   object        
 6   date_added    8807 non-null   datetime64[ns]
 7   release_year  8807 non-null   int64         
 8   rating        8807 non-null   object        
 9   duration      8807 non-null   object        
 10  listed_in     8807 non-null   object        
 11  description   8807 non-null   object        
 12  added_year    8807 non-null   int32         
 13  added_month   8807 non-null   int32         
dtypes: datetime64[ns](1), int32(2), int64(1), object(10)
memory usage: 894.6+ KB
None
  show

In [10]:
type_trend = df.groupby(['added_year','type']).size().unstack().fillna(0)
print(type_trend)

type         Movie  TV Show
added_year                 
2000           0.0     98.0
2008           1.0      1.0
2009           2.0      0.0
2010           1.0      0.0
2011          13.0      0.0
2012           3.0      0.0
2013           6.0      4.0
2014          19.0      4.0
2015          56.0     17.0
2016         253.0    165.0
2017         839.0    325.0
2018        1237.0    388.0
2019        1424.0    575.0
2020        1284.0    594.0
2021         993.0    505.0


In [11]:
# Split multiple genres in 'listed_in'
genres = df['listed_in'].str.split(', ')
all_genres = genres.explode()
top_genres = all_genres.value_counts().head(10)
print(top_genres)

listed_in
International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1351
Documentaries                869
Action & Adventure           859
TV Dramas                    763
Independent Movies           756
Children & Family Movies     641
Romantic Movies              616
Name: count, dtype: int64


In [12]:
top_countries = df['country'].value_counts().head(10)
print(top_countries)

country
United States     2818
India              972
Unknown            831
United Kingdom     419
Japan              245
South Korea        199
Canada             181
Spain              145
France             124
Mexico             110
Name: count, dtype: int64


In [13]:
ratings_count = df['rating'].value_counts()
print(ratings_count)

rating
TV-MA        3207
TV-14        2160
TV-PG         863
R             799
PG-13         490
TV-Y7         334
TV-Y          307
PG            287
TV-G          220
NR             80
G              41
TV-Y7-FV        6
Not Rated       4
NC-17           3
UR              3
74 min          1
84 min          1
66 min          1
Name: count, dtype: int64


In [15]:
# Separate movies vs TV Shows
movies = df[df['type']=='Movie'].copy()

# Remove non-numeric durations and convert to float
movies['duration_min'] = pd.to_numeric(movies['duration'].str.replace(' min',''), errors='coerce')

tv_shows = df[df['type']=='TV Show'].copy()
tv_shows['num_seasons'] = pd.to_numeric(tv_shows['duration'].str.replace(' Season','').str.replace('s',''), errors='coerce')

# Check
print(movies[['title','duration','duration_min']].head())
print(tv_shows[['title','duration','num_seasons']].head())

                               title duration  duration_min
0               Dick Johnson Is Dead   90 min          90.0
6   My Little Pony: A New Generation   91 min          91.0
7                            Sankofa  125 min         125.0
9                       The Starling  104 min         104.0
12                      Je Suis Karl  127 min         127.0
                   title   duration  num_seasons
1          Blood & Water  2 Seasons            2
2              Ganglands   1 Season            1
3  Jailbirds New Orleans   1 Season            1
4           Kota Factory  2 Seasons            2
5          Midnight Mass   1 Season            1


In [16]:
scripts/netflix_queries.sql

NameError: name 'scripts' is not defined

In [18]:
# -----------------------------
# Netflix Analytics Full Pipeline
# -----------------------------
import pandas as pd
import sqlite3
import os

# -----------------------------
# Step 1: Load Netflix Dataset
# -----------------------------
df = pd.read_csv('Users/yash./netflix_titles.csv')

# Fill missing values
df['director'] = df['director'].fillna('Unknown')
df['cast'] = df['cast'].fillna('Unknown')
df['country'] = df['country'].fillna('Unknown')
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df['rating'] = df['rating'].fillna('Unknown')
df['duration'] = df['duration'].fillna('Unknown')

# Extract year & month added
df['added_year'] = df['date_added'].dt.year
df['added_month'] = df['date_added'].dt.month

# -----------------------------
# Step 2: Save to SQLite
# -----------------------------
os.makedirs('outputs', exist_ok=True)
conn = sqlite3.connect('outputs/netflix.db')
df.to_sql('netflix', conn, if_exists='replace', index=False)
conn.close()

print("✅ Dataset cleaned and saved to SQLite database: outputs/netflix.db")

# -----------------------------
# Step 3: Run SQL Queries & Save Results
# -----------------------------
conn = sqlite3.connect('outputs/netflix.db')

# Read all SQL queries from file
with open('scripts/netflix_queries.sql', 'r') as f:
    sql_queries = f.read().split(';')  # split by semicolon

# Run each query and save results
os.makedirs('outputs/query_results', exist_ok=True)
for i, query in enumerate(sql_queries):
    query = query.strip()
    if query:
        df_result = pd.read_sql_query(query, conn)
        # Save to CSV
        output_file = f'outputs/query_results/query_{i+1}.csv'
        df_result.to_csv(output_file, index=False)
        print(f"✅ Query {i+1} result saved to {output_file}")
        print(df_result.head(), "\n")  # Optional: preview

conn.close()
print("🎯 All queries executed and results saved.")

FileNotFoundError: [Errno 2] No such file or directory: 'Users/yash./netflix_titles.csv'